In [2]:
import os
import cv2
import numpy as np
import seaborn as sns
from tqdm import tqdm
from tensorflow.keras.applications import EfficientNetB3
import pickle as pkl
import matplotlib.pyplot as plt

labels = ['covid','non-covid']

# Resize and HSV convert Function for images

In [2]:
def resize(S, dest):
    '''
        1) Making a list of images
        2) Making a list of labels
        3) Resizing the images to SxSx3
        4) Converting the lists into numpy arrays

        params : 
            S -> Resultant Size after Resizing

        return : 
            numpy_array -> contains the images (n, S, S, 3)
            scenes      -> contains the clases (n,)
    '''
    array_of_imgs = []
    to_be_deleted = []
    Y = []

    path_to_each_image = []
    for dirs in labels:
        for images in sorted(os.listdir(dest + '/' + dirs)):
            path_to_each_image.append('/'+ dirs + '/' + images)

    for sub_path in tqdm(path_to_each_image):
        path = dest + sub_path
        img = cv2.imread(path)

        try :
            # resize to SxSx3
            hsvImg = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
            new_img = cv2.resize(hsvImg, (S,S), interpolation = cv2.INTER_AREA)
        except:
            to_be_deleted.append(path)
            continue
        
        # inserting into the array
        array_of_imgs.append(new_img)
        # inserting into labels
        Y.append(path.split('/')[-2])

    # convert lists to numpy arrays
    numpy_array = np.array(array_of_imgs)
    scenes = np.array(Y)

    return numpy_array, scenes

In [3]:
images, scenes = resize(300,"X-ray")

100%|██████████| 9544/9544 [05:16<00:00, 30.16it/s]


In [4]:
scenes

array(['covid', 'covid', 'covid', ..., 'non-covid', 'non-covid',
       'non-covid'], dtype='<U9')

# Dumping HSV images and scenes

In [5]:
def dump_into_pkl(data, name):
    '''
        dumps the data into a pkl file
        
        params : 
            data -> whatever you want to dump
            name -> name of the file
    '''
    outfile = open(name,'wb')
    pkl.dump(data, outfile)
    outfile.close()
    
    print(name + " dumped")

In [6]:
dump_into_pkl(images,"hsvImages.pkl")
dump_into_pkl(scenes,"labels.pkl")

hsvImages.pkl dumped
labels.pkl dumped


# Function to load file

In [ ]:
def load_from_pkl(name):
    infile = open(name, 'rb')
    X = pkl.load(infile)
    infile.close()
    return X

# Load Images, labels

In [ ]:
images = load_from_pkl("hsvImages.pkl")
labels = load_from_pkl("labels.pkl")

In [ ]:
print(images.shape,"\n",labels.shape)

# EfficientNet-B3 Import

In [ ]:
effnetB3 = EfficientNetB3(weights = "imagenet", input_shape = (300,300,3) ,include_top = False, pooling = 'avg')
effnetB3.summary()

# Freezing Layers

In [ ]:
effnetB3.trainable = False

In [ ]:
features.shape
dump_into_pkl(features,"features.pkl")